In [1]:
print("OK!")

OK!


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\anaconda\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "18f0e181-e414-42e8-81f4-947f0ccccca3"
# PINECONE_API_ENV = "gcp-starter"

In [3]:
from pathlib import Path

def load_pdf(data):
    # Get the path to the parent directory of the notebook
    parent_dir = Path().resolve().parent
    
    # Define the path to the data directory
    data_dir = parent_dir / data
    
    # Check if the directory exists
    if not data_dir.exists() or not data_dir.is_dir():
        print("Directory not found or not a directory:", data_dir)
        return None  # or handle the error as needed
    
    # Load the PDFs
    loader = DirectoryLoader(str(data_dir),
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

# Call the function
extracted_data = load_pdf("data")


In [4]:
#Extract data from the PDF
# def load_pdf(data):
#     loader = DirectoryLoader(data,
#                     glob="*.pdf",
#                     loader_cls=PyPDFLoader)
    
#     documents = loader.load()

#     return documents

In [ ]:
# extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [35]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [36]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [ ]:
text_chunks

In [6]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()

In [38]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [13]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medical-chatbot"

In [14]:
from pinecone import ServerlessSpec

# cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
# region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud='aws', region='us-east-1')

In [39]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [42]:
def generate_embeddings(text_chunks, embeddings):
    """
    Generates embeddings for each text chunk using the provided Hugging Face model.

    Args:
        text_chunks: A list of text chunks to be embedded.
        embeddings: The Hugging Face SentenceTransformer model object.

    Returns:
        A list of embedding vectors, one for each text chunk.
    """
    embeddings_list = []
    for chunk in text_chunks:
        if isinstance(chunk, str):
            # If chunk is already a string, use it directly
            text = chunk
        else:
            # If chunk is not a string, convert it to string
            text = str(chunk)

        # Assuming embeddings provides a SentenceTransformer model, use encode method
        embedding = embeddings.embed_query(text)
        embeddings_list.append(embedding)
    return embeddings_list


In [47]:
def generate_embeddings(text_chunks, embeddings, max_chunks=100):
  """
  Generates embeddings for each text chunk using the provided Hugging Face model.

  Args:
      text_chunks: A list of text chunks to be embedded.
      embeddings: The Hugging Face SentenceTransformer model object.
      max_chunks: The maximum number of chunks to process (default: 100).

  Returns:
      A list of embedding vectors, one for each text chunk (up to max_chunks).
  """
  embeddings_list = []
  for i, chunk in enumerate(text_chunks):
    if i >= max_chunks:
      break  # Stop processing after max_chunks are handled

    if isinstance(chunk, str):
      text = chunk
    else:
      text = str(chunk)

    embedding = embeddings.embed_query(text)  # Assuming embeddings uses encode method
    embeddings_list.append(embedding)
  return embeddings_list


In [48]:
processed_chunks = generate_embeddings(text_chunks, embeddings, 100)

In [49]:
print(len(processed_chunks))

100


In [52]:
print(processed_chunks)

[[-0.0279291570186615, 0.06060413643717766, -0.08440717309713364, -0.009406833909451962, 0.0024342555552721024, -0.04193342104554176, -0.04426931217312813, 0.032947685569524765, 0.03695882856845856, 0.025193708017468452, 0.02686537429690361, -0.016500812023878098, 0.037774890661239624, -0.007846037857234478, 0.002514052903279662, 0.023551028221845627, -0.022629637271165848, -0.038086991757154465, 0.010459236800670624, 0.05144298076629639, 0.06256800889968872, 0.1350513994693756, 0.15767090022563934, -0.015276595950126648, 0.027286343276500702, -0.002317767357453704, -0.11386751383543015, -0.01606353372335434, 0.028157047927379608, -0.0371222086250782, 0.12745192646980286, 0.0881897509098053, 0.0392892062664032, 0.002688197186216712, 0.12168089300394058, 0.024278363212943077, 0.004768990911543369, 0.028982939198613167, 0.04612548276782036, 0.05556335672736168, 0.041300877928733826, -0.03614059090614319, -0.0317675955593586, 0.02712761051952839, -0.01325594075024128, -0.08286283165216446

In [ ]:

for batch in tqdm(processed_chunks):
    index.upsert(batch)

In [55]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name="medical-chatbot"

index = pc.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7034}},
 'total_vector_count': 7034}

In [53]:


# docsearch = pc.from_texts(text_data, embeddings, index_name=index_name)
# print("Embeddings created and stored in index:", index_name)

AttributeError: 'Pinecone' object has no attribute 'from_texts'

In [58]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

In [70]:
que = "What are Allergies"
# Define the vector to check
target_vector = embeddings.embed_query(que)
print(target_vector)

[0.018147878348827362, -0.007762169931083918, 0.016550710424780846, -0.02258743718266487, 0.08301927149295807, 0.042130425572395325, 0.047391340136528015, 0.00995915848761797, -0.037392206490039825, 0.07155649363994598, 0.0373200960457325, -0.06228749826550484, 0.01792127825319767, 0.014870431274175644, -0.036336690187454224, 0.08066538721323013, -0.0334295779466629, -0.045790936797857285, -0.0021369382739067078, -0.05767640471458435, -0.01406110916286707, 0.09954630583524704, -0.008254222571849823, -0.005863514728844166, -0.05176488682627678, 0.06957109272480011, -0.03749403357505798, -0.09890786558389664, -0.05588378384709358, -0.03587565943598747, -0.04290854558348656, -0.023026974871754646, 0.0023226940538734198, 0.05676267296075821, -0.04027223214507103, 0.002483945805579424, -0.08057363331317902, 0.020133936777710915, 0.0219374168664217, 0.007606519386172295, -0.02682218886911869, 0.023933125659823418, -0.06894037872552872, -0.03708416223526001, -0.07588568329811096, 0.0623272992

In [103]:

index = pc.Index(index_name)

results= index.query(
    namespace="real",
    vector=target_vector,
    top_k=3,
    include_values=True
)



In [104]:
print(results)

{'matches': [{'id': '1372',
              'score': 0.682538509,
              'values': [0.0354718082,
                         -0.0110478336,
                         0.0751323,
                         0.0080577489,
                         0.113756679,
                         0.0377156027,
                         0.113712057,
                         0.0840440691,
                         -0.0305278674,
                         0.0913646594,
                         0.0113536706,
                         -0.0614714622,
                         0.0239176359,
                         0.0503650531,
                         -0.0131362798,
                         0.0664992481,
                         -0.0492045283,
                         -0.060187459,
                         -0.0266645942,
                         -0.0484286584,
                         0.00717460923,
                         0.0825395361,
                         -0.0148052545,
                         -0.0382491

In [100]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

AttributeError: 'Pinecone' object has no attribute 'from_existing_index'

In [122]:
from langchain.vectorstores import Pinecone

In [ ]:
vectorstore = Pinecone(index, )

In [105]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [106]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [109]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [115]:
docsearch = pc.from_documents(index_name, embeddings)

AttributeError: 'Pinecone' object has no attribute 'from_documents'

In [120]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.as_retriever(embeddings, search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

AttributeError: 'Index' object has no attribute 'as_retriever'

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back that occur when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


KeyboardInterrupt: 